<a href="https://colab.research.google.com/github/bob8dod/NLP_SelfStudying/blob/main/Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Similarity of Vector

## Cosin Similarity


-1 ~ 1 : 1에 가까울 수록 유사도 UP  
문서1 : 저는 사과 좋아요  
문서2 : 저는 바나나 좋아요  
문서3 : 저는 바나나 좋아요 저는 바나나 좋아요  
[ 바나나 사과 저는 좋아요]


In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

# 코사인 유사도 return
def cos_sim(A,B):
    return dot(A,B) / (norm(A)*norm(B))

In [ ]:
# 각각의 BoW
doc1=np.array([0,1,1,1])
doc2=np.array([1,0,1,1])
doc3=np.array([2,0,2,2])

print(cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
print(cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
print(cos_sim(doc2, doc3)) #문서2과 문서3의 코사인 유사도

0.6666666666666667
0.6666666666666667
1.0000000000000002


### Recomandation System using Cosin Similarity

데이터 첨부 안함 그냥 코드만 보시길

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer #TF-IDF
from sklearn.metrics.pairwise import linear_kernel #코사인 유사도

In [ ]:
data = pd.read_csv('현재 movies_metadata.csv의 파일 경로', low_memory=False)
data = data.head(20000)

In [ ]:
#TFIDF에서는 NULL값이 들어가면 오류, Null값 체크 후 조정
print(data['overview'].isnull().sum())
data['overview'] = data['overview'].fillna('')

In [ ]:
#overview에 대한 tf-idf 수행
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

In [ ]:
#코사인 유사도 사용
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
#영화의 타이틀과 인덱스를 가진 테이블 생성
# 이 테이블의 용도는 영화의 타이틀을 입력하면 인덱스를 리턴하기 위함
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head())

In [ ]:
#선택한 영화에 대해서 코사인 유사도를 이용하여, 가장 overview가 유사한 10개의 영화를 찾아내는 함수
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scored = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scored]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return data['title'].iloc[movie_indices]

In [ ]:
get_recommendations('The Dark Knight Rises')
# 영화 다크 나이트 라이즈와 overview가 유사한 영화들
# 가장 유사한 영화가 출력되는데, 영화 다크 나이트가 첫번째고, 그 외에도 전부 배트맨 영화를 찾아낸 것을 확인

## Jaccard similarity

합집합에서 교집합의 비율을 구한다면 두 집합 A와 B의 유사도를 구할 수 있다는 것

In [ ]:
# 두 문서 모두에서 등장한 단어는 apple과 banana 2개
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"
# 토큰화를 수행
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()
# 토큰화 결과
print(tokenized_doc1)
print(tokenized_doc2)

['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
['apple', 'banana', 'coupon', 'passport', 'love', 'you']


In [ ]:
# 문서1과 문서2의 합집합
union = set(tokenized_doc1).union(set(tokenized_doc2))
print(union)
# 문서1과 문서2의 교집합
intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))
print(intersection)
# 자카드 유사도가 계산 (교집합의 원소 개수 / 합집합의 원소 개수)
print('자카드 유사도: ',len(intersection)/len(union))

{'like', 'banana', 'watch', 'likey', 'card', 'passport', 'you', 'holder', 'coupon', 'love', 'apple', 'everyone'}
{'banana', 'apple'}
자카드 유사도:  0.16666666666666666


## Levenshtein Distance

두개의 문장을 2차원 배열로 나타내어서 각 문장의 각 글자들을 비교하여  
몇개의 글자가 다른지 구분하는 방법

### Recomandation System using Levenshtein Distance

In [ ]:
#공개된 자료 없음